In [1]:
# Install Spacy and Beautiful Soup
!pip install beautifulsoup
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install textblob


    Linking successful
    /usr/local/lib/python2.7/dist-packages/en_core_web_sm -->
    /usr/local/lib/python2.7/dist-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [2]:
# Import all the necessary package
import nltk ; print ("NLTK", nltk.__version__)
import BeautifulSoup as bs ; print ("Beatiful Soup", bs.__version__)
from bs4 import BeautifulSoup
from nltk import word_tokenize
import numpy as np
nltk.download('punkt')
import codecs; print ("Codecs", codecs.sys.version)
import urllib2; print ("URLLIB2", urllib2.__version__)
from requests import get; 
import requests; print ("Requests", requests.__version__)
import spacy; print ("Spacy", spacy.__version__)
from nltk.stem import WordNetLemmatizer
print ("WordNetLemmatizer", nltk.version_info)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from nltk.corpus import stopwords 
from textblob import TextBlob
from collections import defaultdict 

import sys; print ("Python version:", sys.version)
nltk.download('stopwords')
import os
print os.environ
print sys.version

('NLTK', '3.2.5')
('Beatiful Soup', '3.2.1')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
('Codecs', '2.7.15rc1 (default, Nov 12 2018, 14:31:15) \n[GCC 7.3.0]')
('URLLIB2', '2.7')
('Requests', '2.18.4')
('Spacy', '2.0.18')
('WordNetLemmatizer', sys.version_info(major=2, minor=7, micro=15, releaselevel='candidate', serial=1))
('Python version:', '2.7.15rc1 (default, Nov 12 2018, 14:31:15) \n[GCC 7.3.0]')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'CLOUDSDK_CONFIG': '/content/.config', 'CUDA_VERSION': '9.2.148', 'PATH': '/usr/local/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin', 'HOME': '/root', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'LANG': 'en_US.UTF-8', 'TERM': 'xterm-color', 'SHELL': '/bin/bash', 'CUDA_

In [0]:
 # Scrape IMDB for list of documentaries
url = 'https://www.imdb.com/search/title?title_type=documentary&explore=genres'
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')


In [0]:
# Grab each of the URLs for all of the documentaries
movie_urls = []
for data in html_soup.find_all('div', class_='lister-item-image float-left'):
    for a in data.find_all('a'):
        link = a.get('href').split("?")
        movie_urls.append(link[0]) #for getting link
        #print(a.text) #for getting text between the link

In [0]:
# Grab all of the reviews for each movie
review_list = []

for movie in movie_urls:
  
  review_url = 'https://www.imdb.com' + movie + 'reviews'
  #print review_url
  response = get(review_url)
  #print(response.text)
  review_soup = BeautifulSoup(response.text, 'html.parser')

  for data in review_soup.find_all('div', class_='text show-more__control'):
    review_list.append(data.text) #for getting text between the link

In [6]:
# Get total number of reviews
len(review_list)

827

In [0]:
# Extract all of the noun chunks from each review and print them out
# Also lemmatize while we are at it.

lemma_reviews = []
sent_reviews = []
nlp = spacy.load('en_core_web_sm')
for review in review_list:
  doc = nlp(review)
  lemma_reviews.append(" ".join([token.lemma_ for token in doc]))
  for chunk in doc.noun_chunks:
    sent = TextBlob(chunk.text)
    if (sent.polarity != 0):
      sent_reviews.append((chunk.text, sent.polarity))

In [8]:
# Remove stop words
stop_words = set(stopwords.words('english')) 
stop_words.add("film")
stop_words.add("movie")
stop_words.add("documentary")
stop_words.add("go")
stop_words.add("see")
stop_words.add("watch")
stop_words.add("pron") # Spacy lemmatizer throws these in for pronouns.

# Vectorize corpus
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(lemma_reviews)

k = 10
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

# Print out clusters.
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print


Top terms per cluster:
Cluster 0:  banksy  art  artist  street  mcqueen  guetta  thierry  exit  shop  fashion
Cluster 1:  make  show  one  like  good  people  would  life  great  get
Cluster 2:  russian  doping  russia  dope  athlete  sport  us  putin  bryan  rodchenkov
Cluster 3:  moore  food  gun  eat  spurlock  people  fast  mcdonald  black  michael
Cluster 4:  porn  girl  rocco  woman  industry  make  star  court  life  get
Cluster 5:  paul  walker  football  gerrard  player  legend  steven  love  great  family
Cluster 6:  carrey  whitney  jim  andy  kaufman  houston  life  know  moon  singer
Cluster 7:  climb  climber  meru  climbing  mountain  alex  story  rock  el  chin
Cluster 8:  band  coldplay  love  music  fan  album  friendship  like  masterpiece  full
Cluster 9:  story  nicholas  family  three  bourdin  triplet  year  identical  nature  twin


In [9]:
# Create a dictionary of lists. One list for each cluster.
sent_dict = defaultdict(list)

for idx in range(0, X.shape[0]):
  pred = model.predict(X[idx])
  key = str(pred[0])
  sent_dict[key].append(sent_reviews[idx][1])
print sent_dict


defaultdict(<type 'list'>, {'1': [-0.016666666666666663, 0.2, 0.1, 0.375, 0.16666666666666666, -0.2, 0.1, 0.1, 0.375, 0.25, -0.3, 0.5, 0.375, 0.4, 0.13636363636363635, 0.1, 0.11363636363636363, -0.125, 0.25, 0.5, 0.24675324675324675, 0.3, 0.2, 0.4, 0.25, 0.5, -1.0, 0.75, 0.30000000000000004, 0.1, 0.4166666666666667, 0.5, 0.1, 0.8, -0.15555555555555559, 0.8, 0.5, 0.5, 0.1, 0.6, -0.2, -0.07777777777777777, 0.3, 0.1, 0.375, 0.1, 0.5, -0.08333333333333333, 0.5, 0.125, 0.05000000000000002, -0.1, 0.5, 0.6, 0.5, 0.7, 0.3333333333333333, 0.21428571428571427, 0.8, 0.9, -0.3, 0.30000000000000004, 0.2, 0.1, -0.16666666666666666, -0.1, 0.8, 0.2, 0.3333333333333333, 0.16666666666666666, 0.15000000000000002, -0.25, 0.25, 0.10000000000000002, 0.25, 0.1, 0.375, 0.8, -0.16666666666666666, 0.7, -0.3333333333333333, 0.5, 0.8, -0.07142857142857142, 0.375, 0.5, 0.75, 0.25, 0.8, 0.25, 0.5, 0.7, 0.2, 0.15625, 0.1, 0.35, 0.5, 0.6, 0.5, 0.5, 0.13636363636363635, 0.5, 0.5, 0.5, 0.2, 0.1, 0.8, 0.5, 1.0, 0.6, -0.

In [0]:
# Sort the sentiment of each review 
sentiment = sorted(sent_reviews,key=lambda x:(-x[1],x[0]))


In [11]:
# Print out highest sentiment
print sentiment[0:4]

[(u'"Pitch Perfect', 1.0), (u'(BEST', 1.0), (u', what a wonderful concept', 1.0), (u'A Wonderful World', 1.0)]


In [12]:
# Print out the lowest sentiment
print sentiment[-4:]

[(u'this insane performance', -1.0), (u'this terrible death toll', -1.0), (u'those two awful instances', -1.0), (u'totally a horrible experience', -1.0)]


In [13]:
# Print out sentiment statistics by cluster.
for key in sent_dict:
  print "Stats for cluster", key , "High: ", max(sent_dict[key]), "Low: ", min(sent_dict[key]), "Mean: ", np.mean(sent_dict[key])
  #print "Highest sentiment value:", max(sent_dict[key])
  #print "Lowest sentiment value:", min(sent_dict[key])
  #print "Mean sentiment:", np.mean(sent_dict[key])

Stats for cluster 1 High:  1.0 Low:  -1.0 Mean:  0.2201287987563043
Stats for cluster 0 High:  0.6 Low:  -0.8 Mean:  0.18239670868347335
Stats for cluster 3 High:  1.0 Low:  -0.8 Mean:  0.24065553494124922
Stats for cluster 2 High:  0.8 Low:  -0.5 Mean:  0.28833333333333333
Stats for cluster 5 High:  1.0 Low:  -1.0 Mean:  0.1931872294372294
Stats for cluster 4 High:  1.0 Low:  -0.7 Mean:  0.2558697778125489
Stats for cluster 7 High:  1.0 Low:  -0.321428571429 Mean:  0.2702631909153649
Stats for cluster 6 High:  0.78 Low:  -1.0 Mean:  0.13757548007548007
Stats for cluster 9 High:  1.0 Low:  -0.5 Mean:  0.2639869505494506
Stats for cluster 8 High:  0.9 Low:  -0.1875 Mean:  0.33622557997557995
